In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from html_to_markdown import convert_to_markdown
from IPython.display import display, Markdown

In [ ]:
from nlpready._mlabc import readx_suba_papers_csv
papers=list(readx_suba_papers_csv('suba_j.csv'))

In [ ]:
from collections import Counter
counts = Counter()
for p in papers:
    if p.issn:
        counts[p.issn]+=1

In [ ]:
from nlpready._selenium import Selenium
from nlpready._issn import DATA

In [ ]:
p = None
for idx, p1 in enumerate(reversed(papers)):
    if p1.issn:
        if p1.issn in {"0253-9772", '1756-2651'}:
            continue
    if (p1.issn not in DATA) and p1.year > 2000 and counts[p1.issn] > 20:
        p = p1
        break
print(idx, len(papers), len(counts), len(DATA))
p.issn in DATA

In [ ]:
f"https://doi.org/{p.doi}"

In [ ]:
p

In [ ]:
web = Selenium(timeout=10,headless=False)
markdown = web.run(p.doi, DATA[p.issn])
Markdown(markdown)

----

## EPMC

In [ ]:
from bs4 import BeautifulSoup
from io import StringIO, BytesIO
from html_to_markdown import convert_to_markdown
from IPython.display import HTML, Markdown

In [ ]:
from nlpready.epmc.epmc import EPMC
from nlpready.epmc.utils import Events, PMCEvents, PMCTAGS

In [ ]:
DONE= {'PMC5486124', 'PMC5447075', 'PMC5853234', 'PMC5490910', 'PMC5490927', 'PMC5823009', 'PMC5399074',
      'PMC5575511', 'PMC5387099', 'PMC5405513', 'PMC5412409', 'PMC5462045', 'PMC5444443', 'PMC5435427'}

In [ ]:

for p in reversed(papers):
    if p.pmcid and p.pmcid not in DONE and p.year > 2010:
        break
p

In [ ]:
f"https://doi.org/{p.doi}"

In [ ]:
CSS="""
<style>
.email { color:orange; }
.title-group { color: blue; font-size: 14pt; }
figure { border: solid orange 1px; padding: 0.5em; }
.missing { font-size:larger; color: red; }
.sc { color: red; font-weight: bold; }
</style>
"""

In [ ]:
x =  EPMC.from_pmcid(p.pmcid)
html = x.html()
if x.missing:
    print(x.missing)
HTML(CSS + html)

In [ ]:
# x.save_content('saved.xml')

In [ ]:
from nlpready.epmc.utils import convert_html
Markdown(convert_to_markdown(convert_html(html,dict(caption='p', label='div'))))

In [ ]:
x.save_content("xxx.xml", pretty=True)